Import libraries and key to access twitter. Keys and tokens can be gerated with an twitter account at https://developer.twitter.com/en/portal/dashboard

In [1]:
from keys import *
import tweepy
import pandas as pd
import requests

create client to access the twitter API
Following tutorial: https://www.kirenz.com/post/2021-12-10-twitter-api-v2-tweepy-and-pandas-in-python/twitter-api-v2-tweepy-and-pandas-in-python/

In [2]:

client = tweepy.Client( bearer_token=bearer_token, 
                        consumer_key=consumer_key, 
                        consumer_secret=consumer_secret, 
                        access_token=access_token, 
                        access_token_secret=access_token_secret, 
                        return_type = requests.Response,
                        wait_on_rate_limit=True)


How to create queries https://medium.com/@robguilarr/making-queries-to-twitter-api-on-tweepy-66afeb7184a4

ToDO so schreiben das queries aus Suchwörtern aus zwei Listen abgefragr werden und Duplicate entfernt werden.
Anschließend müssen Duplicate entfernt werden

Bahn Bezug:
regio
Zug
sbahn
ubahn
Stadtbahn
Regionalbahn
Straßenbahn
Tram
Bus
Fernbus
Stadtbus
bahn
ÖPNV
ICE
IC
RE

Fahrrad Bezug:
Fahrrad
rad
bike
ebike
Elektrorad
Elektrobike
Elektrofahrrad
pedelec
velo
Mountainbike
Trekkingbike
Radl

In [3]:
def get_tweets(query, max_results):
    tweets = client.search_recent_tweets(query=query, 
                                        tweet_fields=['attachments',
                                                        'author_id',
                                                        'context_annotations',
                                                        'conversation_id',
                                                        'created_at',
                                                        'geo',
                                                        'id',
                                                        'in_reply_to_user_id',
                                                        'lang',
                                                        'public_metrics',
                                                        'referenced_tweets',
                                                        'text'],
                                         max_results=max_results)
    return tweets


In [4]:
keywords = {'bahn_context':['regio',
                            'Zug',
                            'sbahn',
                            'ubahn',
                            'Stadtbahn',
                            'Regionalbahn',
                            'Straßenbahn',
                            'Tram',
                            'Bus',
                            'Fernbus',
                            'Stadtbus',
                            'bahn',
                            'ÖPNV',
                            'ICE',
                            'IC',
                            'RE'],
            'bike_context':['Fahrrad',
                            'call a bike',
                            'rad',
                            'bike',
                            'ebike',
                            'Elektrorad',
                            'Elektrobike',
                            'Elektrofahrrad',
                            'pedelec',
                            'velo',
                            'Mountainbike',
                            'Trekkingbike',
                            'Radl']
           }

In [5]:
# 192 Permutationen

max_results = 100 # maximum is 100 due to ESSENTIAL access 
tweets_data = pd.DataFrame()
for first_keyword in keywords['bahn_context']:
    for second_keyword in keywords['bike_context']:
        query = first_keyword + ' ' + second_keyword
        tweets = get_tweets(query,max_results)
        # Save data as dictionary
        tweets_dict = tweets.json()
        # check if tweets where found for this query
        if 'data' in tweets_dict:
            # Extract "data" value from dictionary
            new_tweet_data = tweets_dict['data']
            # Transform to pandas Dataframe
            df_new_tweet_data = pd.json_normalize(new_tweet_data)
            df_new_tweet_data['query'] = query
            # concat dataframes with previously loaded data
            tweets_data = pd.concat([tweets_data, df_new_tweet_data])
            print(len(df_new_tweet_data), 'results for query', query, 'returned')           
        else:
            print('0 results for query', query, 'returned')

print(len(tweets_data),'tweets where found with', tweets_data['id'].nunique(), 'unique tweet_ids')

3 results for query regio Fahrrad returned
0 results for query regio call a bike returned
4 results for query regio rad returned
3 results for query regio bike returned
0 results for query regio ebike returned
0 results for query regio Elektrorad returned
0 results for query regio Elektrobike returned
0 results for query regio Elektrofahrrad returned
0 results for query regio pedelec returned
5 results for query regio velo returned
0 results for query regio Mountainbike returned
0 results for query regio Trekkingbike returned
0 results for query regio Radl returned
74 results for query Zug Fahrrad returned
0 results for query Zug call a bike returned
76 results for query Zug rad returned
2 results for query Zug bike returned
0 results for query Zug ebike returned
0 results for query Zug Elektrorad returned
0 results for query Zug Elektrobike returned
0 results for query Zug Elektrofahrrad returned
2 results for query Zug pedelec returned
9 results for query Zug velo returned
0 results 

4 results for query IC Fahrrad returned
0 results for query IC call a bike returned
48 results for query IC rad returned
22 results for query IC bike returned
0 results for query IC ebike returned
0 results for query IC Elektrorad returned
0 results for query IC Elektrobike returned
0 results for query IC Elektrofahrrad returned
0 results for query IC pedelec returned
2 results for query IC velo returned
0 results for query IC Mountainbike returned
0 results for query IC Trekkingbike returned
0 results for query IC Radl returned
6 results for query RE Fahrrad returned
33 results for query RE call a bike returned
99 results for query RE rad returned
99 results for query RE bike returned
100 results for query RE ebike returned
0 results for query RE Elektrorad returned
0 results for query RE Elektrobike returned
0 results for query RE Elektrofahrrad returned
0 results for query RE pedelec returned
100 results for query RE velo returned
6 results for query RE Mountainbike returned
0 resul

In [6]:
save_filename = pd.Timestamp.today().strftime('%Y-%m-%d') + '_tweets_DB_Fahrrad'
tweets_data.to_pickle('Tweets/' + save_filename)

In [7]:
tweets_data['lang'].value_counts()

de     945
en     802
fr     185
tr      37
ja      12
lv      12
nl       6
cs       6
in       5
it       4
und      3
ko       3
hi       3
es       2
pt       1
mr       1
sv       1
Name: lang, dtype: int64

In [8]:
#likes = client.get_liking_users(1512355733555326982)


In [9]:
#likes.json()

- Auf Twitter gibt es ca. 20 Tweets pro Tag zum Thema Bahn & Fahrrad. Reicht das?
- Nur Zugriff auf Daten der letzten 7 Tage

https://towardsdatascience.com/collect-data-from-twitter-a-step-by-step-implementation-using-tweepy-7526fff2cb31https://towardsdatascience.com/collect-data-from-twitter-a-step-by-step-implementation-using-tweepy-7526fff2cb31